# Movie Recommendation System 

## Importing and Exploring

In [40]:
# Importing libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors

In [41]:
# Loading all the datasets
df_movie = pd.read_csv('movie.csv').set_index('movieId')
df_rating = pd.read_csv('rating.csv')

In [42]:
# Browsing the dataset
df_movie.sample(n=10)

,title,genres
movieId,,
30776,Born to Dance (1936),Comedy|Musical
6157,Daredevil (2003),Action|Crime
70175,Winged Creatures (Fragments) (2008),Crime|Drama
4386,Cats & Dogs (2001),Children|Comedy
42491,"When You Comin' Back, Red Ryder? (1979)",Drama
95352,Jekyll & Hyde (1990),Horror
32387,"Sword of Doom, The (Dai-bosatsu tôge) (1966)",Action|Drama
25897,Thousands Cheer (1943),Comedy|Drama|Musical
5137,Scratch (2001),Documentary


## Preprocessing Data

### Preprocessing df_movie

In [43]:
df_movie['year'] = df_movie['title'].str[-6:]
df_movie['year'] = df_movie['year'].str[1:5]

def convert(data):
    number = preprocessing.LabelEncoder()
    data['year'] = number.fit_transform(data['year'])
    data=data.fillna(0) # fill holes with default value
    return data
convert(df_movie)
df_main = pd.merge(df_movie, df_rating, on=['movieId'])
df_main = df_main.set_index('movieId')

### Preprocessing df_final

In [44]:
table= df_main.pivot_table(index='movieId', aggfunc={'rating' : np.mean})
df_final = pd.merge(table, df_movie, on=['movieId'])
df_final = df_final.reset_index()
categories = ['Adventure', 'Animation','Children','Comedy','Romance','Horror','Sci-Fi','Western','Fantasy','Crime','Drama','Thriller','Musical','War','Mystery','Documentary']
for category in categories:
    df_final[category] = df_final['genres'].str.contains(category)
    df_final[category] = np.where(df_final[category] == True, 1, 0)

### Preprocessing df_final_focus

In [45]:
df_final_focus = df_final.set_index('movieId', inplace=True)
df_final_focus = df_final.drop( ['title' , 'genres', 'rating'], axis = 1)

## Training the Model

In [46]:
# Find Exact Name
name = df_movie['title'].str.contains('Spider')
movie_name = df_movie[name]
movie_name

,title,genres,year
movieId,,,
2823,"Spiders Part 1: The Golden Lake, The (Die Spin...",Action|Adventure|Drama,38
4051,Horrors of Spider Island (Ein Toter Hing im Ne...,Horror|Sci-Fi,79
4238,Along Came a Spider (2001),Action|Crime|Mystery|Thriller,120
5349,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,121
5356,"Giant Spider Invasion, The (1975)",Horror|Sci-Fi,94
6197,Spider (2002),Drama|Mystery,121
6430,Ziggy Stardust and the Spiders from Mars (1973),Documentary|Musical,92
6786,Kiss of the Spider Woman (1985),Drama,104
6848,Kingdom of the Spiders (1977),Horror|Sci-Fi,96


In [47]:
movie_name = 'Amazing Spider-Man, The (2012)'
fav_movie = df_final.loc[df_final['title'] == movie_name]
fav_movie_focus = fav_movie.drop( ['title' , 'genres', 'rating'], axis = 1)
fav_movie_focus

,year,Adventure,Animation,Children,Comedy,Romance,Horror,Sci-Fi,Western,Fantasy,Crime,Drama,Thriller,Musical,War,Mystery,Documentary
movieId,,,,,,,,,,,,,,,,,
95510,131,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [48]:
nbrs = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10).fit(df_final_focus)

In [49]:
distance, index = nbrs.kneighbors(fav_movie_focus)

## Getting Recommendations

In [50]:
for i in range(9):
    i = index[0][i+1]
    print(df_movie.iloc[i]['title'])

Bag of Bones (2011)
Skeletons (2010)
La Soufrière - Warten auf eine unausweichliche Katastrophe (1977)
Bad Chicken (2013)
Ideal Husband, An (1947)
Two Little Boys (2012)
English Teacher, The (2013)
Zero Charisma (2013)
Lottery Ticket (2010)
